In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs

## Överblick

**Målet med dagens övning  modifiera våra neuroner samt träningsloop till att kunna lösa binära klassificeringsproblem.**

Vi kommer skapa vår egen data mha scikit-learn.

In [ ]:
coordinates, labels = make_blobs(n_samples=[500,500], 
                                 n_features=3, random_state=41)

# visualisera de nyss skapade blobsen

fig = plt.figure(figsize = (10, 8))
ax = plt.axes(projection ="3d")

ax.scatter3D(coordinates[:,0],
             coordinates[:,1],
             coordinates[:,2], 
             c=labels)

ax.set_title("Class Divisions");

## Dela upp data i train/test och utför nödvändiga transformationer

Som vi diskuterat behöver vi kunna uppskatta prestanda som kan förväntas uppnås i  en skarp, produktionssatt situation. Att enbart evaluera på träningsdata är därför inte lämpligt. Vi delar därför upp vårt dataset i en train- och en testsplit.

Vi väljer att göra det i proportionen 90/10.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(coordinates,
                                                    labels,
                                                    test_size=0.1, 
                                                    random_state=42)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

**Skala data**

Nu behöver vi på enklaste sätt bara skala om kolumnerna så att samtliga värden, till absolutbeloppet, inte blir särskilt mycket större än 1. Vi ser i vår illustrering ovan att så inte är fallet nu - utan att samtliga koordinater antar större värden än så.

Ett supersimpelt trick för att åstadkomma detta är att helt enkelt dela respektive kolumn, med det (till absolutbeloppet) högsta värdet. Då kommer samtliga värden skalas ner, och det högsta värdet i respektive kolumn vara (i absolut värde) 1.

**Kom dock ihåg att det är superviktigt att vi endast använder statistik från train split när vi transformerar, annars riskerar vi informationsläckage!**

Detta var anledningen till att vi delade upp datasetet i train/test-split innan vi började med dessa invasiva ingrepp.

In [ ]:
from sklearn.preprocessing import StandardScaler

for i in range(3):                               # iterera över alla vår tre coordinater
    
    highest_value = max(np.abs(x_train[:,i]))    # hitta det, till absolutbeloppet, högsta värdet i aktuella kolumn i x_train
    
    
    # dela nu aktuell kolumn i x_train med det nyfunna högsta värdet
    
    x_train[:,i] = x_train[:,i] / highest_value 
    
    # dela nu även motsvarande kolumn i x_test med SAMMA nyfunna högsta värde (från x_train)
    
    x_test[:,i] = x_test[:,i] / highest_value

Vi kan nu återigen visualisera vår data, som en sanity check. Lägg märke till skalan.

In [ ]:
fig = plt.figure(figsize = (5, 5))
ax = plt.axes(projection ="3d")

ax.scatter3D(x_train[:,0],
             x_train[:,1],
             x_train[:,2], 
             c=y_train)

ax.set_title("Train split");

plt.show()

fig = plt.figure(figsize = (5, 5))
ax = plt.axes(projection ="3d")

ax.scatter3D(x_test[:,0],
             x_test[:,1],
             x_test[:,2], 
             c=y_test)

ax.set_title("Test split");

**Omvandla till Tensor**

Nu när siffrorna ser bra ut återstår det att omvandla till datatypen Tensor (optimal för PyTorch)

In [ ]:
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.FloatTensor).reshape([-1,1])

x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor).reshape([-1,1])

training_set = list(zip(x_train, y_train))             # lägg ihop träningsdatan så att vi direkt kan skicka in i dataloader
test_set = list(zip(x_test, y_test))                   # ditto för testdatan

## Skapa Neurons med Sigmoid-aktiveringsfunktion i PyTorch

In [ ]:
import torch.nn as nn

class neuron(nn.Module):
    def __init__(self, input_size):
        super(neuron, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)                  
        return x

In [ ]:
input_size = 3

model = neuron(3)
model

**KONTROLLFRÅGA: hur många parametrar har den här modellen?**

Låt oss testa vad vår, hitills otränade, modell spottar ut sig för output

In [ ]:
sample = 200                                     # välj här vilken sample som helst, ange värde mellan 0-899

sample_features = x_train[sample]                # extrahera features
sample_class = y_train[sample]                   # extrahera class (1 eller 0)

model_prediction = model(sample_features)        # predicta outcome

In [ ]:
print('True class              :', sample_class.item())
print('Vår models predict      :', model_prediction.item())

Notera att, för vilken sample du än väljer, så spottar vår modell ut sig värden mellan 0 och 1! Sigmoid-aktiveringsfunktion säkerställer detta.


Det kan kanske också se ut som att vår modell verkar predicta någorlunda bra so far, eftersom att både True outcome och våra predictions endast antar värden mellan 0 och 1 och således inte ligger så långt ifrån varandra - men alla våra predictions är helt random at this point, var så säkra. Testa själv med olika samples.Sannolikheterna vi spottar ur nonsens.

Och det är ju inte så konstigt - för vi har ju inte tränat något än!

## Träna

Observera att vi i koden nedan lagt till ytterliggare en sektion kod under vår epoch-loop. Denna ansvarar för att evaluera koden. 

Ignorera den för nu - du kommer få jobba med den strax.

In [ ]:
from torch.optim import SGD
from torch.utils.data import DataLoader

In [ ]:
input_size = 3                       
batch_size = 16

epochs = 10                    # antal loopar genom dataloader vi låter vår modell träna på vårt dataset.
learning_rate = 0.5          # hur stora steg gradient descent tar.

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
#    initera modell, loss_function, optimizer & dataloader


model = neuron(input_size)
optimizer = SGD(model.parameters(), lr = learning_rate)
loss_function = torch.nn.BCELoss()

train_dataloader = DataLoader(training_set,                 
                              batch_size = batch_size,       
                              shuffle=True)


# ----------------------------------------------------------------------------------------------------------------
#    träna


batch_losses = []
evaluation_losses = []

for i in range(epochs):
    
    model.train()
    
    for batch in train_dataloader:
        
        y_true = batch[1]
        input_features = batch[0]
        
        y_pred=model(input_features)
        loss=loss_function(y_pred, y_true)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        batch_loss = loss.item()
        batch_losses.append(batch_loss)

# ----------------------------------------------------------------------------------------------------------------
#   evalueringssektion 

    model.eval()
    
    y_true = y_test
    input_features = x_test
    
    y_pred = model(input_features)
    loss = loss_function(y_pred, y_true)
    
    evaluation_loss = loss.item()
    evaluation_losses.append(evaluation_loss)
    
        
plt.plot(batch_losses, label='train loss')
plt.plot(np.arange(1, epochs+1)*(len(train_dataloader)), evaluation_losses, label='test loss')
plt.legend()
plt.xlabel('batch')
plt.show()

## Uppgifter

**1)**

Vi ska nu utforska sambandet mellan number of epochs och learning rate. Vi ska testa kombinationer av olika värden på dessa parametrar, och se hur det påverkar vår loss.

Låt oss testa följande värden:

epochs = [1,5,10,100,200]

learning_rates = [0.0001, 0.001, 0.01, 0.1]

Testa varje möjlig kombination av dessa värden, dvs

    for epoch in epochs:
    
        for learning_rate in learning_rates:
        
            run training loop
            plot(loss)
        
*Vad ser du för kurvor? Vad upptäcker du för samband? Does it make sense?*

**2)**

Testa nedan kodsnutt för ett antal olika sampels. Ser det bättre ut nu?

**Varför tror du att logistisk regression kallas just regression, även fast vi använder det till klassificering?**

In [ ]:
sample = 90                                     # välj här vilken sample som helst, ange värde mellan 0-899

sample_features = x_test[sample]                # extrahera features
sample_class = y_test[sample]                   # extrahera class (1 eller 0)

model_prediction = model(sample_features)        # predicta outcome

In [ ]:
print('True class              :', sample_class.item())
print('Vår models predict      :', model_prediction.item())

**2)**

Fokusera nu på evalueringssektionen i vår träningskod. Säkerställ att du förstår vad som händer, steg för steg!

**3)**

Nu ska vi visualisera det vår evalueringssektion fångade upp. Ersätt de sista raderna kod (de som plottar) i träningsscellen med följande:

________________________________________________________

plt.plot(batch_losses, label='train loss')
plt.plot(np.arange(1, epochs+1)*(len(train_dataloader)), evaluation_losses, label='test loss')
plt.legend()
plt.xlabel('batch')
plt.show()
________________________________________________________

Testa nu att köra om din träningsloop för olika kombinationer av epoch och learning rate. 

Vad ser du? Does it make sense?

**4**

Gå tillbaks till början av vår notebook och försök att generera ett dataset som är svårare att klassificera, dvs ett där våra blobs inte går att separera like lätt. Du kan enkelt generera nya dataset genom att ändra på make_blobs-funktionens parameter random_state. 

Hitta ett värde på random_state som ger oss ett svårare dataset - träna sedan en ny modell (med olika värden på epoch och learning rate) och tolka det du ser.